In [1]:
import os
import shutil
import pathlib
import random
import math
import sys

In [2]:
full_dataset_path = './dataset/training_hu'
split_dataset_path = './dataset/split_hu'
validation_split = 0.2
test_split = 0.02
seed = 19011997

In [3]:
dataset_path = pathlib.Path(full_dataset_path)
split_path = pathlib.Path(split_dataset_path)

In [4]:
if os.path.isdir(split_dataset_path):
    shutil.rmtree(split_dataset_path)

os.makedirs(split_path/'validation')
os.makedirs(split_path/'testing')


In [5]:
shutil.copytree(dataset_path, split_path/'training')

WindowsPath('dataset/split_hu/training')

In [6]:
class_names = os.listdir(full_dataset_path)
print(class_names)

['healthy', 'unhealthy']


In [7]:
for name in class_names:
    os.makedirs(split_path/'validation'/name)
    os.makedirs(split_path/'testing'/name)

In [8]:
files = list(split_path.glob('training/*/*.jpg'))
count = len(files)
print(count)

17728


In [9]:
def get_filename_tail(file_path):
  head, tail2 = os.path.split(file_path)
  _, tail1 = os.path.split(head)
  return os.path.join(tail1, tail2)

def get_class(file_path):
  head, _ = os.path.split(file_path)
  _, clas = os.path.split(head)
  return clas.__str__()

In [10]:
random.seed(seed)
random.shuffle(files)

In [11]:
def move_file(file_path, subset):
    subfilename = get_filename_tail(file_path)
    shutil.move(file_path, split_path/subset/subfilename)

def balanced_copy(subset, count):
    total_count = count
    balance = {}
    for name in class_names:
        balance[name] = 0
    while count > 0:
        max_balance = max(balance.values())
        i = 0
        flag = all(x == max_balance for x in balance.values())
        file_path = ''
        if flag:
            file_path = files.pop()
        while not flag:
            file = files[i]
            clas = get_class(file)
            if balance[clas] < max_balance:
                file_path = files.pop(i)
                break
            i = i + 1

        clas = get_class(file_path)
        balance[clas] = balance[clas] + 1
        move_file(file_path, subset)
        count -= 1
        sys.stdout.write(str(count))
        sys.stdout.flush()
        sys.stdout.write("\r")


def unbalanced_copy(subset, count):
    total_count = count
    while count > 0:
        move_file(files.pop(), subset)
        count -= 1
        sys.stdout.write(str(count))
        sys.stdout.flush()
        sys.stdout.write("\r")

In [12]:
count_test = int(count * test_split)
count_validation = int(count * validation_split)

balanced_copy('testing', count_test)
unbalanced_copy('validation', count_validation)